In [8]:
from flightanalysis.fc_json import FCJson
from flightplotting.plots import plotdtw, plotsec
from flightanalysis import Section

with open("FlightAnalysis/tests/test_inputs/manual_F3A_P21_21_09_24_00000052.json", "r") as f:
    fcj = FCJson.parse_fc_json(f)

direct_sec = Section.from_csv("FlightAnalysis/tests/test_inputs/test_log_00000052_section.csv")

/home/tom/miniconda3/envs/pyflightcoach/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/tom/miniconda3/envs/pyflightcoach/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [2]:

#plotdtw(fcj.sec, fcj.schedule.manoeuvres).show()
#man = fcj.schedule.manoeuvres[7]
#plotdtw(man.get_data(fcj.sec), man.elements)

In [10]:
import plotly.express as px

px.line(fcj.sec.data, y=["brvr", "brvp", "brvy"])

In [2]:
from flightanalysis import get_schedule
from flightanalysis.schedule.element import get_rates

rates = get_rates(fcj.sec)


sched = get_schedule("F3A", "P21").scale_distance(rates["distance"])

rates

{flightanalysis.schedule.element.LoopEl: 1.0799170046637068,
 flightanalysis.schedule.element.LineEl: 4.3317702848327935,
 flightanalysis.schedule.element.SnapEl: 315.23027793701283,
 flightanalysis.schedule.element.StallTurnEl: 180.06205676237374,
 flightanalysis.schedule.element.SpinEl: 78.71089492781934,
 'speed': 29.269122645586474,
 'distance': 163.36709286341568}

In [3]:
from flightanalysis import Section
# this is needed to get just the scored sequence from a section
sequence = fcj.sec.subset(sched.manoeuvres[0].get_data(fcj.sec).data.index[0],sched.manoeuvres[-2].get_data(fcj.sec).data.index[-1]).remove_labels()

dist, aligned = Section.align(sequence, sched.create_raw_template(sequence.get_state_from_index(0).direction, 1.5*rates["speed"], rates["distance"]), 2)
dist


31654.90532102191

In [4]:
sum(fcj._dists)

43141.43317106192

In [6]:
from plotly.subplots import make_subplots
import flightplotting.templates
from flightplotting.traces import dtwtrace
import numpy as np

man = sched.manoeuvres[1]

fig = make_subplots(
    1,
    2,
    specs=[[{'is_3d': True}, {'is_3d': True}]]
)
fig.update_layout(template="flight3d", showlegend=False, width=800, height=500)


fig.add_traces(
    dtwtrace(man.get_data(aligned), man.elements), 
    rows=list(np.full(len(man.elements)+2, 1)),
    cols=list(np.full(len(man.elements)+2, 1))
)
fig.add_traces(
    dtwtrace(man.get_data(fcj.sec), man.elements),  
    rows=list(np.full(len(man.elements)+2, 1)),
    cols=list(np.full(len(man.elements)+2, 2))
)
fig.show()

